In [6]:
pip install line-bot-sdk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent
import requests

app = Flask(__name__)

# 👉 替換為你的 LINE Channel 設定
configuration = Configuration(access_token='+PO90hgIpgGHJFRNLcplCKRsfJTxS1HeltWYjmTMo302xeK0Nl79y4Jf+R2fx8JyfV5kh35OEbfivRl2+Kl1danQ5Yl5z86t38iktFxsCrtK3YcRgaG1L7A+wJaDSdM9d9KFyb4PION0Z7HTIiDoFgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('c762e444aa124c1b0eff8f40013f5cc4')


@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. 請確認 access token / secret")
        abort(400)

    return 'OK'


@handler.add(MessageEvent)
def handle_message(event):
    try:
        if not isinstance(event.message, TextMessageContent):
            return

        user_text = event.message.text.strip()
        reply_text = ""

        # 🎯 判斷是否是股票查詢指令：「查股價 股票代碼」
        if user_text.startswith("查股價"):
            parts = user_text.split()
            if len(parts) == 2 and parts[1].isdigit():
                stock_code = parts[1]
                reply_text = fetch_stock_price(stock_code)
            else:
                reply_text = "📌 請輸入正確格式，例如：查股價 2330"
        else:
            reply_text = "🤖 請輸入指令：「查股價 股票代碼」\n例如：查股價 2330"

        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

    except Exception as e:
        print("❌ 回覆過程出錯：", str(e))
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 處理過程中發生錯誤，請稍後再試。")]
                )
            )


# 🏦 查詢台灣股價資料（TWSE 非官方 API）
def fetch_stock_price(stock_code):
    url = f"https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_{stock_code}.tw"
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        if not data.get("msgArray"):
            return f"❌ 查無代碼 {stock_code}，請確認是否正確"

        info = data["msgArray"][0]
        name = info.get("n", stock_code)
        price = info.get("z", "無報價")
        high = info.get("h", "—")
        low = info.get("l", "—")
        open_price = info.get("o", "—")
        volume = info.get("v", "—")

        return (
            f"📊【{name}】{stock_code}\n"
            f"💰 最新成交價：{price}\n"
            f"📈 開盤：{open_price}｜最高：{high}｜最低：{low}\n"
            f"📦 成交量：{volume}"
        )
    except Exception as e:
        return f"⚠️ 查詢過程出錯：{str(e)}"

# 🚪 啟動 Flask 應用（記得修改 port）
if __name__ == "__main__":
    app.run(port=6000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6000
Press CTRL+C to quit
